In [6]:
import math
import pandas as pd
import matplotlib.pyplot as plt

## Important functions for calculations

In [7]:
import math
def sind(t):
    return math.sin(math.radians(t))
def cosd(t):
    return math.cos(math.radians(t))
def tand(t):
    return math.tan(math.radians(t))
def asind(t):
    return math.degrees(math.asin(t))
def acosd(t):
    return math.degrees(math.acos(t))
def atand(t):
    return math.degrees(math.atan(t))
def decl(n):
    de = 23.45*sind((360*(284+n))/365)
    return de
def ha(st):
    return 15*(12-st)
def theta(beta , delta , gamma , omega , phi):
    ca = sind(phi)*(sind(delta)*cosd(beta)+cosd(delta)*cosd(gamma)*cosd(omega)*sind(beta)) + cosd(phi)*(cosd(delta)*cosd(omega)*cosd(beta)-sind(delta)*cosd(gamma)*sind(beta)) + cosd(delta)*sind(gamma)*sind(omega)*sind(beta)
    return acosd(ca)
def theta_z(delta , gamma , omega , phi):
    return theta(0, delta , gamma ,omega ,phi)
def rb(n, st , beta ,gamma, phi):
    ca = cosd(theta(beta, decl(n), gamma, ha(st),phi))
    caz = cosd(theta_z(decl(n), gamma, ha(st),phi))
    rb = ca/caz
    return rb
def rd(beta):
    return 0.5*(1+cosd(beta))
def rr(beta, rohg):
    return (rohg/2)*(1-cosd(beta))
def thetadash(ng , na , theta):
    return asind(sind(theta)*(na/ng))
def I_total(Ibh,Idh , rb , rd , rr):
    return (Ibh*rb + Idh*rd + (Ibh+Idh)*rr)
def roh1(theta , thetad):
    return pow(sind(thetad-theta),2)/pow(sind(thetad+theta),2)
def roh2(theta , thetad):
    return pow(tand(thetad-theta),2)/pow(tand(thetad+theta),2)
def tau1(roh1 , m):
    return (1-roh1)/(1+roh1*(2*m-1))
def tau2(roh2 , m):
    return (1-roh2)/(1+roh2*(2*m-1))
def tau_a(theta_d,k,delC):
    return math.exp(-(k*delC)/cosd(theta_d))
def alpha(theta):
    up = 0.0106944*(theta-90)*(1-0.254*pow((cosd(theta)/cosd(85)),4*(theta-85)))
    down = pow((theta/95.406),0.9775)-1
    if type(up/down) == complex:
        return 0
    else:
        return up/down
def tau_alp_b(tau,alpha,roh_d):
    return (tau*alpha)/(1-(1-alpha)*roh_d)
def tau_alp_d(theta ,ng,na,m,k,delC,roh_d):
    td = thetadash(ng,na,theta)
    roh_1 = roh1(theta,td)
    roh_2 = roh2(theta,td)
    tau_1 =  tau1(roh_1,m)
    tau_2 =  tau2(roh_2,m)
    taua = tau_a(td,k,delC)
    alp = alpha(theta)
    return tau_alp_b((taua*0.5*(tau_1+tau_2)),alp,roh_d)
def S(Ibh, Idh, rb, rd, rr, tau_b , tau_d):
    return (Ibh*rb*tau_b + (Idh*rd+(Ibh+Idh)*rr)*tau_d)
def fin_eff(U,K,cpst,L):
    m = pow((U/(K*cpst)),0.5)
    return ((math.exp(m*L)-math.exp(-m*L))/(math.exp(m*L)+math.exp(-m*L)))/(m*L)
def collector_eff(U,W,L,F,Do,cb,Di,hci):
    up = 1/U
    down1 = 1/(U*(2*L*F+Do))
    down2 = 1/cb
    down3 = 1/(hci*math.pi*Di)
    return up/(W*(down1+down2+down3))
def hear_removal_factor(mw,cw,U,fd):
    return (mw*cw/U)*(1-math.exp((-U*fd)/(mw*cw)))

## Constants

In [8]:
st = [6,7,8,9,10,11,12,13,14,15,16,17,18]
st_t = [6,7,8,9,10,11,12,13,14,15,16,17,18,'Daily']
I = [
    [[0,0,147,300,426,509,537,509,426,300,147,0,0],[0,0,56,77,91,98,98,98,91,77,56,0,0]],
    [[0,42,216,391,502,593,621,593,502,391,216,42,0],[0,25,70,91,98,105,105,105,98,91,70,28,0]],
    [[0,105,293,453,593,649,712,649,593,453,293,105,0],[0,49,84,98,105,105,105,105,105,98,84,49,0]],
    [[0,160,356,523,663,747,767,747,663,523,356,160,0],[0,70,91,98,105,112,112,112,105,98,91,70,0]],
    [[42,209,398,565,705,788,816,788,705,565,398,209,42],[42,77,98,105,105,112,112,112,105,105,98,77,42]],
    [[63,230,412,586,705,788,816,788,705,586,412,230,63],[49,77,98,105,112,112,112,112,112,105,98,77,49]],
    [[56,223,405,565,684,767,795,767,684,565,405,223,56],[42,77,98,105,105,112,112,112,105,105,98,77,42]],
    [[28,188,363,530,670,753,781,753,670,530,363,188,28],[35,70,91,105,105,112,112,112,105,105,91,70,35]],
    [[0,119,307,474,607,698,726,698,607,474,307,119,0],[0,63,84,98,105,105,112,105,105,98,84,63,0]],
    [[0,49,223,384,516,593,628,593,516,384,223,49,0],[0,35,77,91,98,105,105,105,98,91,77,35,0]],
    [[0,0,147,300,419,495,523,495,419,300,147,0,0],[0,0,63,84,91,98,98,98,91,84,63,0,0]],
    [[0,0,112,265,391,460,495,460,391,265,112,0,0],[0,0,49,77,91,98,98,98,91,77,49,0,0]]
]

Twi = [14.8, 16.1, 19.4, 22.7, 24.3, 27.5, 28.5, 29.3, 27.9, 26, 21.4, 17.6]
Ta_r = [
    [8.8, 19.7],
    [9.8, 21.2],
    [11.9, 24.4],
    [14.8, 28.5],
    [17.5, 32.2],
    [20.7, 34.9],
    [22, 34.7],
    [22.3, 34.5],
    [20.6, 33.1],
    [18.1, 30.6],
    [14, 25.5],
    [10.3, 21.1]   
]
DR = [98,93,84,71,56,39,23,11,3,0,3,10,21]
avg_d = [17,47,75,105,135,162,198,228,258,288,318,334]
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

### Variables to be edited

In [9]:
gamma = 0
beta = 30
phi = 30
n_o_c = 1
k = 0.0068
ng = 1.526
na = 1
delC = 4
roh_g = 0.2
U = 5
K = 385
cpst = 0.5/1000
Do= 11/1000
Di= 10/1000
cb = 35
W = 150/1000
mw = 0.015
cw = 4180
hci = 300
twh=55
vw = 250
Twh=55
thta_diff = 20
if n_o_c == 1 :
    roh_d = 0.15
elif n_o_c == 2:
    roh_d = 0.22
elif n_o_c == 3: 
    roh_d = 0.24

# Cheking the manual samples

# Working on average days

In [51]:
m = int(input("Enter the month\n"))
n = avg_d[m-1]
print("Day number =" , n)

Enter the month
5
Day number = 135


In [52]:
I_total_m = []
S_m = []
Q_a
for i in range(10):
    beta = 10*i
    theta_in = []
    theta_z_in = []
    rb_in = []
    I_total_in = []
    thetadash_in = []
    roh1_in = []
    roh2_in = []
    tau1_in = []
    tau2_in = []
    tau_r = []
    tau_a_in = []
    tau = []
    alpha_in = []
    tau_a_b_in = []
    S_in = []
    qu = []

    # DIFFUSE
    rd_in = rd(beta)
    rr_in = rr(beta, roh_g)
    tau_a_d_in = tau_alp_d(thta_diff, ng, na, n_o_c, k, delC, roh_d)

    # TANK
    L = (W-Do)/2
    F = fin_eff(U,K,cpst,L)
    F_d = collector_eff(U,W,L,F,Do,cb,Di,hci)
    FR = hear_removal_factor(mw,cw,U,F_d)
    for i in range(len(st)):
        theta_in.append(theta(beta,decl(n),gamma,ha(st[i]),phi))
        theta_z_in.append(theta_z(decl(n),gamma,ha(st[i]),phi))
        if rb(n,st[i],40,20,30) < 0:
            rb_in.append(0)
        else:
            rb_in.append(rb(n,st[i],40,20,30))
        I_total_in.append(I_total(I[m-1][0][i],I[m-1][1][i],rb_in[i],rd(beta),rr(beta,0.2)))
        thetadash_in.append(thetadash(ng,na,theta_in[i]))
        roh1_in.append(roh1(theta_in[i],thetadash_in[i]))
        roh2_in.append(roh2(theta_in[i],thetadash_in[i]))
        tau1_in.append(tau1(roh1_in[i],n_o_c))
        tau2_in.append(tau1(roh2_in[i],n_o_c))
        tau_r.append((tau1_in[i]+tau2_in[i])*0.5)
        tau_a_in.append(tau_a(thetadash_in[i],k,delC))
        tau.append(tau_a_in[i]*tau_r[i])  
        alpha_in.append(alpha(theta_in[i]))
        tau_a_b_in.append(tau_alp_b(tau[i],alpha_in[i],roh_d))
        S_in.append(S(I[m-1][0][i],I[m-1][1][i],rb_in[i], rd_in, rr_in, tau_a_b_in[i], tau_a_d_in))
    ta = Ta_r[m-1][1] - DR[0]*(Ta_r[m-1][1]-Ta_r[m-1][0])/100
    tw_n = [Twi[m-1]]
    if FR*(S_in[0]-U*(Twi[m-1]-ta)) < 0 :
        qu.append(0)
    else:
        qu.append(FR*(S_in[0]-U*(Twi[m-1]-ta)))

    for i in range(1,len(st)): 
        two = tw_n[i-1]+(qu[i-1]/(mw*cw))
        tw_n.append((two+tw_n[i-1])/2)
        ta = Ta_r[m-1][1] - DR[i]*(Ta_r[m-1][1]-Ta_r[m-1][0])/100
        if FR*(S_in[i]-U*(tw_n[i]-ta)) > 0: 
            qu.append(FR*(S_in[i]-U*(tw_n[i]-ta)))
        else:
            qu.append(0)
    I_total_daily = sum(I_total_in[1:len(I_total_in)])*(3600/pow(10,6))
    S_daily = sum(S_in[1:len(S_in)])*(3600/pow(10,6))
    Q_daily = sum(qu[1:len(qu)])*(3600/pow(10,6))
    Ac = (vw*cw*(Twh-Twi[m-1]))/(Q_daily*pow(10,6))
    width = Ac/2
    No_of_tubes = int((Ac/2)/width)
    I_total_in.append(I_total_daily)
    S_in.append(S_daily)
    qu.append(Q_daily)
    
    I_total_m.append(I_total_in)
    S_m.append(S_in)
    Q_m.append(qu)    

## Putting data in data frames

In [63]:
I_avg = pd.DataFrame(data= {
    "ST":st_t,
    "Beta = 0" : I_total_m[0],
    "Beta = 10" : I_total_m[1],
    "Beta = 20" : I_total_m[2],
    "Beta = 30" : I_total_m[3],
    "Beta = 40" : I_total_m[4],
    "Beta = 50" : I_total_m[5],
    "Beta = 60" : I_total_m[6],
    "Beta = 70" : I_total_m[7],
    "Beta = 80" : I_total_m[8],
    "Beta = 90" : I_total_m[9]
})
S_avg = pd.DataFrame(data= {
    "ST":st_t,
    "Beta = 0" : S_m[0],
    "Beta = 10" : S_m[1],
    "Beta = 20" : S_m[2],
    "Beta = 30" : S_m[3],
    "Beta = 40" : S_m[4],
    "Beta = 50" : S_m[5],
    "Beta = 60" : S_m[6],
    "Beta = 70" : S_m[7],
    "Beta = 80" : S_m[8],
    "Beta = 90" : S_m[9],
})
Q_avg = pd.DataFrame(data= {
    "ST":st_t,
    "Beta = 0" : Q_m[0],
    "Beta = 10" : Q_m[1],
    "Beta = 20" : Q_m[2],
    "Beta = 30" : Q_m[3],
    "Beta = 40" : Q_m[4],
    "Beta = 50" : Q_m[5],
    "Beta = 60" : Q_m[6],
    "Beta = 70" : Q_m[7],
    "Beta = 80" : Q_m[8],
    "Beta = 90" : Q_m[9],
})


In [64]:
print(f"Total solar radiation at {months[m-1]} at different tilt angles")
I_avg

Total solar radiation at February at different tilt angles


,ST,Beta = 0,Beta = 10,Beta = 20,Beta = 30,Beta = 40,Beta = 50,Beta = 60,Beta = 70,Beta = 80,Beta = 90
0,6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,7,215.299690,215.211575,214.949908,214.522638,213.942748,213.227858,212.399690,211.483407,210.506850,209.499690
2,8,549.096814,548.999584,548.710847,548.239377,547.599499,546.810655,545.896814,544.885743,543.808162,542.696814
3,9,775.221635,775.262654,775.384465,775.583366,775.853315,776.186109,776.571635,776.998181,777.452785,777.921635
4,10,871.360280,871.527394,872.023661,872.834000,873.933791,875.289616,876.860280,878.598058,880.450150,882.360280
5,11,944.695394,944.958220,945.738712,947.013155,948.742825,950.875169,953.345394,956.078446,958.991281,961.995394
6,12,929.943107,930.248471,931.155285,932.635996,934.645614,937.123076,939.993107,943.168502,946.552778,950.043107
7,13,851.162549,851.425374,852.205866,853.480309,855.209980,857.342323,859.812549,862.545600,865.458435,868.462549
8,14,697.555883,697.722997,698.219264,699.029603,700.129394,701.485219,703.055883,704.793661,706.645753,708.555883
9,15,530.797541,530.838560,530.960370,531.159272,531.429221,531.762014,532.147541,532.574086,533.028690,533.497541


In [65]:
print(f"Incident solar flux at {months[m-1]} at different tilt angles")
S_avg

Incident solar flux at February at different tilt angles


,ST,Beta = 0,Beta = 10,Beta = 20,Beta = 30,Beta = 40,Beta = 50,Beta = 60,Beta = 70,Beta = 80,Beta = 90
0,6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,7,50.496011,95.475407,126.026741,145.390910,156.971887,163.620696,167.146626,168.583769,168.462790,166.967403
2,8,322.083152,383.755064,418.287243,437.385886,447.814752,453.127525,455.098427,454.494966,451.412172,445.303002
3,9,570.850127,614.752577,637.919166,650.522769,657.410882,660.806987,661.594801,659.883810,655.094244,645.578147
4,10,694.478282,722.507227,737.450749,745.945496,750.900982,753.381957,753.566627,751.231025,745.315132,732.992759
5,11,775.618915,796.258519,807.740597,814.713847,818.987631,820.891723,820.407500,817.101493,809.201489,792.202681
6,12,769.583182,786.090380,795.586802,801.399483,804.761110,805.878747,804.547389,799.743067,788.654068,764.919121
7,13,699.256992,713.567503,721.797115,726.482119,728.628738,728.365706,725.067962,716.976644,700.198900,666.550998
8,14,557.295857,569.974347,577.128759,580.617743,581.147383,578.583459,571.906900,558.794546,534.669196,491.009414
9,15,394.813176,405.687164,411.449639,413.193016,411.211738,405.076497,393.530691,374.212225,343.279223,295.404292


In [66]:
print(f"Rate of useful energy gain at {months[m-1]} at different tilt angles")
Q_avg

Rate of useful energy gain at February at different tilt angles


,ST,Beta = 0,Beta = 10,Beta = 20,Beta = 30,Beta = 40,Beta = 50,Beta = 60,Beta = 70,Beta = 80,Beta = 90
0,6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,7,19.485281,57.614393,83.512811,99.927851,109.745065,115.381272,118.370209,119.588477,119.485923,118.218281
2,8,253.400279,304.390904,332.788527,348.423653,356.932400,361.245544,362.815255,362.262522,359.652699,354.516791
3,9,461.997036,496.200979,514.004135,523.604939,528.824588,531.367185,531.880934,530.408029,526.439584,518.589212
4,10,558.429227,578.021083,588.251787,594.044999,597.449934,599.130834,599.115932,597.163328,592.374226,582.410311
5,11,616.551655,629.217350,636.167984,640.475612,643.187580,644.322641,643.741222,641.032000,634.722836,621.131478
6,12,598.326803,607.061352,612.093811,615.272024,617.119091,617.549108,616.269223,612.381449,603.582217,584.740009
7,13,524.286059,530.863245,534.651731,536.766587,537.521259,536.766345,533.862811,527.320051,513.995874,487.387302
8,14,390.090059,395.061469,397.810455,398.839962,398.198410,395.518542,389.848920,379.270965,360.168565,325.972271
9,15,240.617771,243.891469,245.367593,244.882547,242.134170,236.517470,226.911811,211.430458,187.202859,150.589209
